# Judicial Integrity Analysis - Example Walkthrough

This notebook demonstrates how to use the Judicial Integrity Analysis package
to analyze judges in Arizona (Phase 1 target state).

## Contents
1. Data Collection
2. Preprocessing
3. Corruption Analysis
4. Ethics Evaluation
5. Sentencing Bias Detection
6. Partisanship Assessment
7. Composite Integrity Summary
8. Visualization
9. Report Generation

In [ ]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configure logging
logging.basicConfig(level=logging.INFO)

# Import modules
from judicial_integrity_analysis.src.data_acquisition import JudicialDataAggregator
from judicial_integrity_analysis.src.preprocessing import JudicialDataPreprocessor
from judicial_integrity_analysis.src.analysis import (
    JudicialIntegrityAnalyzer,
    SentencingDisparityAnalyzer,
)
from judicial_integrity_analysis.src.visualization import JudicialVisualizer
from judicial_integrity_analysis.src.report_generator import ReportGenerator

%matplotlib inline
plt.style.use('default')

## 1. Data Collection

First, we collect judicial data for Arizona. This requires API access to CourtListener.
If you don't have an API token, the client will use unauthenticated access with lower rate limits.

In [ ]:
# Initialize aggregator for Arizona
aggregator = JudicialDataAggregator(
    state='az',
    data_dir='../data',
    # courtlistener_token='your_token_here'  # Optional
)

# Collect data (limit for demo purposes)
raw_data = aggregator.collect_all(max_judges=10)

print(f"Federal judges: {len(raw_data.get('federal_judges', []))}")
print(f"State judges: {len(raw_data.get('state_judges', []))}")
print(f"Disciplinary records: {len(raw_data.get('disciplinary_records', []))}")
print(f"Performance reviews: {len(raw_data.get('performance_reviews', []))}")

## 2. Preprocessing

Clean and normalize the collected data.

In [ ]:
preprocessor = JudicialDataPreprocessor(raw_data)
preprocessor.preprocess_all()

if preprocessor.judges is not None:
    print(f"Preprocessed {len(preprocessor.judges)} judges")
    display(preprocessor.judges.head())

## 3. Analysis

Run the four-dimensional integrity analysis.

In [ ]:
analyzer = JudicialIntegrityAnalyzer(
    judges_df=preprocessor.judges,
    disciplinary_df=preprocessor.disciplinary_records,
    performance_df=preprocessor.performance_reviews,
    sentencing_df=preprocessor.sentencing_data,
)

# Corruption indicators
corruption = analyzer.analyze_corruption_indicators()
print("Corruption Risk Indicators:")
display(corruption.head())

In [ ]:
# Ethics evaluation
ethics = analyzer.analyze_ethics_indicators()
print("Ethics Indicators:")
display(ethics.head())

In [ ]:
# Partisanship assessment
partisanship = analyzer.analyze_partisanship_indicators()
print("Partisanship Indicators:")
display(partisanship.head())

In [ ]:
# Composite integrity summary
summary = analyzer.compute_integrity_summary()
print("Integrity Summary:")
display(summary.head(10))

In [ ]:
# Text report
print(analyzer.generate_text_report())

## 4. Visualization

Create visual dashboards of the analysis results.

In [ ]:
viz = JudicialVisualizer(figsize=(12, 8))

# Risk tier distribution
if not summary.empty:
    fig = viz.plot_risk_tier_distribution(summary)
    plt.show()

In [ ]:
# Integrity scores
if not summary.empty:
    fig = viz.plot_integrity_scores(summary)
    plt.show()

In [ ]:
# Partisanship by appointer
if not partisanship.empty:
    fig = viz.plot_partisanship_by_appointer(partisanship)
    plt.show()

## 5. Report Generation

Generate publication-ready Markdown reports.

In [ ]:
reporter = ReportGenerator(state='az', output_dir='../output/reports/az')

# State summary report
report = reporter.generate_state_summary(
    judges_df=preprocessor.judges if preprocessor.judges is not None else pd.DataFrame(),
    integrity_summary=summary,
)
print(report[:1000] + '\n...')

## Summary

This notebook demonstrated the full Judicial Integrity Analysis workflow:

1. **Data Collection**: Gathered judge data from CourtListener and state sources
2. **Preprocessing**: Normalized and structured heterogeneous data
3. **Corruption Analysis**: Assessed risk from disciplinary actions and financial disclosures
4. **Ethics Evaluation**: Scored based on performance reviews
5. **Bias Detection**: Analyzed sentencing disparities (when data available)
6. **Partisanship Assessment**: Examined appointment backgrounds
7. **Composite Scoring**: Combined all dimensions into integrity summary
8. **Visualization**: Created dashboards and charts
9. **Report Generation**: Produced publication-ready Markdown reports

For command-line usage, see `main.py`. For LLM-assisted auditing, configure `OPENROUTER_API_KEY`.